In [84]:
from re import DEBUG
from flask import Flask, jsonify, request ,render_template, send_file
import pickle
from PIL import Image, ImageDraw
from torchvision.transforms import functional as func
import torch
import base64
import io
import re
import json
from base64 import decodestring
import numpy as np

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

app=Flask(__name__)


@app.route('/predictObject',methods=["POST"])
def predict():
    imagestr=""
#     for file in list(request.files):
#         imagestr=request.files[file]
#     print(request.body())
    data=None
    content_type = request.headers.get('Content-Type')
    if (content_type == 'application/json; charset=utf-8'):
#         try:
        print("running1")
#         print(request.json)
        data = json.loads(request.data.decode("utf-8") )
        print(data)
        print("running2")

    #     data = request.json
    #     print(data)
        imagestr = data['imagefile']
    # process Image
    #     img = Image.fromstring('RGB',(width,height),decodestring(imagestr))
    #     print(imagestr)
    #     imgObj = io.BytesIO(base64.decodebytes(bytes(imagestr,"ascii")))
    #     print(type(imgObj))
    #     img = Image.open(imgObj)
        print("Image received")
        imagestr = re.sub('^data:image/.+;base64,', '', imagestr)
        decoded = base64.b64decode(imagestr)
        img = Image.open(io.BytesIO(decoded))
    #     img = preprocess_image(img, target_size=(224, 224))

    #     img = Image.open((base64.b64decode(imagestr)))
    #     img=Image.open(imagePath)
        # Run the model on GPU if it is available

        image = func.to_tensor(img)
        # image=torch.tensor(image)
        c, h, w = image.shape
    #     img.show()
        # Perform inference
        preds = model([image])[0]
        labels = preds["labels"].cpu().detach().numpy()
        scores = preds["scores"].cpu().detach().numpy()
        boxes = preds["boxes"].cpu().detach().numpy()


        img1 = ImageDraw.Draw(img) 
        result={}
        for label, score, box in zip(labels, scores, boxes):
            # Convert to [top-left-x, top-left-y, width, height]
            # in relative coordinates in [0, 1] x [0, 1]
            if score>= .9 :
                x1, y1, x2, y2 = box
    #             rel_box = [x1 / w, y1 / h, (x2r - x1) / w, (y2 - y1) / h]
                rel_box=[x1, y1, x2, y2]
                name=label if len(COCO_INSTANCE_CATEGORY_NAMES)<label else COCO_INSTANCE_CATEGORY_NAMES[label]
                #result[label]=[name,score]
                img1.rectangle(rel_box,  outline ="blue")
    #             img=img1
        img.show()
        # return send_file(imagePath, mimetype='image/jpg')     
        imgByteArr = io.BytesIO()
        img = np.array(img, dtype=np.uint8)
        img=Image.fromarray(img)
        img.save(imgByteArr, format='jpeg')
        imgByteArr = imgByteArr.getvalue()
        print("Image sending")
        return str(jsonify({"resultant_image" : str(base64.b64encode(imgByteArr))}))
#         except Exception as e:
#             print(e)
#             return "Error"
        
    else:
        print('Content-Type not supported!'+content_type)
        return 'Content-Type not supported!'

# http://10.0.2.2:8080/
if __name__ == '__main__':
    with open('mlmodel.pickle', 'rb') as f:
            model = pickle.load(f)
    app.run(port=2000,debug=True, use_reloader=False)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:2000/ (Press CTRL+C to quit)


running1


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

127.0.0.1 - - [14/Jun/2022 14:20:23] "POST /predictObject HTTP/1.1" 200 -


Image sending
running1


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

127.0.0.1 - - [14/Jun/2022 14:24:30] "POST /predictObject HTTP/1.1" 200 -


Image sending
running1


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

127.0.0.1 - - [14/Jun/2022 14:27:32] "POST /predictObject HTTP/1.1" 200 -


Image sending


In [1]:
from re import DEBUG
from flask import Flask, jsonify, request ,render_template, send_file
import pickle
from PIL import Image, ImageDraw , ImageFont
from torchvision.transforms import functional as func
import torch
import base64
import io
import re
import json
from base64 import decodestring
import numpy as np

font_family = "arial.ttf"
fontsize=30
font = ImageFont.truetype(font_family, fontsize)

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

app=Flask(__name__)


@app.route('/predictObject',methods=["POST"])
def predict():
    imagestr=""
#     for file in list(request.files):
#         imagestr=request.files[file]
#     print(request.body())
    data=None
    content_type = request.headers.get('Content-Type')
    if (content_type == 'application/json; charset=utf-8'):
#         try:
        print("running1")
#         print(request.json)
        data = json.loads(request.data.decode("utf-8") )
        print("running2")

    #     data = request.json
    #     print(data)
        imagestr = data['imagefile']
    # process Image
    #     img = Image.fromstring('RGB',(width,height),decodestring(imagestr))
    #     print(imagestr)
    #     imgObj = io.BytesIO(base64.decodebytes(bytes(imagestr,"ascii")))
    #     print(type(imgObj))
    #     img = Image.open(imgObj)
        print("Image received")
        imagestr = re.sub('^data:image/.+;base64,', '', imagestr)
        decoded = base64.b64decode(imagestr)
        img = Image.open(io.BytesIO(decoded))
    #     img = preprocess_image(img, target_size=(224, 224))

    #     img = Image.open((base64.b64decode(imagestr)))
    #     img=Image.open(imagePath)
        # Run the model on GPU if it is available
        img = img.resize((round(img.size[0]*0.5), round(img.size[1]*0.5)))
        
        image = func.to_tensor(img)
        # image=torch.tensor(image)
        c, h, w = image.shape
    #     img.show()
        # Perform inference
        preds = model([image])[0]
        labels = preds["labels"].cpu().detach().numpy()
        scores = preds["scores"].cpu().detach().numpy()
        boxes = preds["boxes"].cpu().detach().numpy()

        img1 = ImageDraw.Draw(img) 

        result={}
        for label, score, box in zip(labels, scores, boxes):
            # Convert to [top-left-x, top-left-y, width, height]
            # in relative coordinates in [0, 1] x [0, 1]
            if score>= .9 :
                x1, y1, x2, y2 = box
    #             rel_box = [x1 / w, y1 / h, (x2r - x1) / w, (y2 - y1) / h]
                rel_box=[x1, y1, x2, y2]
                name=label if len(COCO_INSTANCE_CATEGORY_NAMES)<label else COCO_INSTANCE_CATEGORY_NAMES[label]
                #result[label]=[name,score]
                img1.rectangle(rel_box,  outline ="blue",width=7 )
                img1.multiline_text((x1, y1), "{} : {:.2f}".format(name,score) ,font=font,  fill=(255, 255, 255))
    #             img=img1
        img.show()
        # return send_file(imagePath, mimetype='image/jpg')     
        imgByteArr = io.BytesIO()
        img = np.array(img, dtype=np.uint8)
        img=Image.fromarray(img)
        img.save(imgByteArr, format='jpeg')
        imgByteArr = imgByteArr.getvalue()
        print("Image sending")
        s= str(json.dumps({"resultant_image" : str(base64.b64encode(imgByteArr))}))
#         print(s)
        return s
#         except Exception as e:
#             print(e)
#             return "Error"
        
    else:
        print('Content-Type not supported!'+content_type)
        return 'Content-Type not supported!'

# http://10.0.2.2:8080/
if __name__ == '__main__':
    with open('mlmodel.pickle', 'rb') as f:
            model = pickle.load(f)
    app.run(port=2000,debug=True, use_reloader=False)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:2000/ (Press CTRL+C to quit)


running1
running2
Image received


127.0.0.1 - - [11/Jul/2022 19:06:10] "POST /predictObject HTTP/1.1" 200 -


Image sending
running1
running2
Image received


127.0.0.1 - - [11/Jul/2022 19:07:35] "POST /predictObject HTTP/1.1" 200 -


Image sending


In [1]:
from re import DEBUG
from flask import Flask, jsonify, request ,render_template, send_file
import pickle
from PIL import Image, ImageDraw , ImageFont
from torchvision.transforms import functional as func
import torch
import base64
import io
import re
import json
from base64 import decodestring
import numpy as np

font_family = "arial.ttf"
fontsize=30
font = ImageFont.truetype(font_family, fontsize)

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

app=Flask(__name__)


@app.route('/predictObject',methods=["POST"])
def predict():
    imagestr=""
#     for file in list(request.files):
#         imagestr=request.files[file]
#     print(request.body())
    data=None
    content_type = request.headers.get('Content-Type')
    if (content_type == 'application/json; charset=utf-8'):
#         try:
        print("running1")
#         print(request.json)
        data = json.loads(request.data.decode("utf-8") )
        print("running2")

    #     data = request.json
    #     print(data)
        imagestr = data['imagefile']
    # process Image
    #     img = Image.fromstring('RGB',(width,height),decodestring(imagestr))
    #     print(imagestr)
    #     imgObj = io.BytesIO(base64.decodebytes(bytes(imagestr,"ascii")))
    #     print(type(imgObj))
    #     img = Image.open(imgObj)
        print("Image received")
        imagestr = re.sub('^data:image/.+;base64,', '', imagestr)
        decoded = base64.b64decode(imagestr)
        img = Image.open(io.BytesIO(decoded))
    #     img = preprocess_image(img, target_size=(224, 224))

    #     img = Image.open((base64.b64decode(imagestr)))
    #     img=Image.open(imagePath)
        # Run the model on GPU if it is available
        img = img.resize((round(img.size[0]*0.5), round(img.size[1]*0.5)))
        
        image = func.to_tensor(img)
        # image=torch.tensor(image)
        c, h, w = image.shape
    #     img.show()
        # Perform inference
        preds = model([image])[0]
        labels = preds["labels"].cpu().detach().numpy()
        scores = preds["scores"].cpu().detach().numpy()
        boxes = preds["boxes"].cpu().detach().numpy()

        img1 = ImageDraw.Draw(img) 

        result={}
        for label, score, box in zip(labels, scores, boxes):
            # Convert to [top-left-x, top-left-y, width, height]
            # in relative coordinates in [0, 1] x [0, 1]
            if score>= .9 :
                x1, y1, x2, y2 = box
    #             rel_box = [x1 / w, y1 / h, (x2r - x1) / w, (y2 - y1) / h]
                rel_box=[x1, y1, x2, y2]
                name=label if len(COCO_INSTANCE_CATEGORY_NAMES)<label else COCO_INSTANCE_CATEGORY_NAMES[label]
                #result[label]=[name,score]
                img1.rectangle(rel_box,  outline ="blue",width=7 )
                img1.multiline_text((x1, y1), "{} : {:.2f}".format(name,score) ,font=font,  fill=(255, 255, 255))
    #             img=img1
        img.show()
        # return send_file(imagePath, mimetype='image/jpg')     
        imgByteArr = io.BytesIO()
        img = np.array(img, dtype=np.uint8)
        img=Image.fromarray(img)
        img.save(imgByteArr, format='jpeg')
        imgByteArr = imgByteArr.getvalue()
        print("Image sending")
        s= str(json.dumps({"resultant_image" : str(base64.b64encode(imgByteArr))}))
#         print(s)
        return s
#         except Exception as e:
#             print(e)
#             return "Error"
        
    else:
        print('Content-Type not supported!'+content_type)
        return 'Content-Type not supported!'

# http://10.0.2.2:8080/
if __name__ == '__main__':
    with open('mlmodel.pickle', 'rb') as f:
            model = pickle.load(f)
    app.run(port=2000,debug=True, use_reloader=False)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:2000/ (Press CTRL+C to quit)


running1
running2
Image received


127.0.0.1 - - [15/Jun/2022 19:38:40] "POST /predictObject HTTP/1.1" 200 -


Image sending
